In [161]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import pandas as pd

from kiwipiepy import Kiwi

from collections import Counter

import time

In [170]:
kiwi = Kiwi()

def count_word(text, word_counts):
    # kiwi.analyze()로 분석 후, 품사가 'NNG'(일반 명사), 'NNP'(고유 명사)인 경우만 추출
    nouns = [token.form for token in kiwi.analyze(text)[0][0] if token.tag.startswith('NN')]
    new_word_counts = Counter(nouns) + word_counts
    return new_word_counts

In [171]:
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))

# url에 포함되어 있는 parameters
# 정렬 : sort={"0":관련도순, "1":최신순, "2":오래된순}
# 기간 : (시작기간)ds=YYYY.MM.DD, (끝기간)de=YYYY.MM.DD
# 유형 : photo={"0":전체, "1":포토, "2":동영상, "3":지면기사, ...}
# mynews=1, office_type=3, office_category=3으로 수정
company_name = "코스피"" ""마감"  # 검색어
date = "2016.02.11"      # 검색기간(하루로 제한 -> 추후 수정 예정)
url = f"https://search.naver.com/search.naver?ssc=tab.news.all&query={company_name}&sm=tab_opt&sort=0&photo=3&field=0&pd=3&ds={date}&de={date}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{date.replace(".","")}to{date.replace(".","")}&is_sug_officeid=0&office_category=3&service_area=0"
driver.get(url)
time.sleep(2)            # 사이트 차단 방지를 위한 멈춤

In [172]:
# 기간 내 기사들의 링크 모두 가져오기

# 모든 기사가 보일때까지 스크롤
len_before_scroll = 0
while (True):
    # 현재 로드된 뉴스 기사 목록을 모두 가져온다 (CSS 선택자는 실제 페이지에 맞게 수정)
    articles = driver.find_elements(By.CSS_SELECTOR, "div.BHQHyn3Flk5rFBSacJkG")

    # 스크롤 후 기사 수
    len_after_scroll = len(articles)

    if len(articles) > 100:
        break

    # 목록의 끝에서 두 번째 항목을 타겟으로 지정 (마지막 항목은 푸터와 겹칠 수 있음)
    if len(articles) > 1:
        target_element = articles[-2]
        
        # JavaScript를 실행하여 해당 요소가 보이도록 스크롤
        driver.execute_script("arguments[0].scrollIntoView()", target_element)

        time.sleep(1)

    # 스크롤 전후 기사 수 비교 (같다면 더이상 스크롤 할 필요 X)
    if len_before_scroll == len_after_scroll:
        break

    len_before_scroll = len_after_scroll

In [173]:
# 네이버 뉴스 주소 가져오기

news_urls = []
news_info = driver.find_elements(By.CLASS_NAME, "sds-comps-horizontal-layout.sds-comps-full-layout.sds-comps-profile.type-basic.size-lg.title-color-g10.hRlTUxpL38wmDKxyAhSU")
for info in news_info:
    news_link = info.find_element(By.CLASS_NAME, "GUWgsNcVrWa67MoYor6N.xR1x3GgF_MIcYPUZNqEu")
    news_urls.append(news_link.get_attribute("href"))

In [ ]:
driver.quit()

In [174]:
def get_news_content(url):
    req = Request(url, headers={'User-Agent': generate_user_agent()})
    page = urlopen(req)
    return BeautifulSoup(page, "html.parser")

In [175]:

word_counts=Counter()
for url in news_urls:
    soup = get_news_content(url)
    time.sleep(1)
    news_title = soup.find("div", class_="media_end_head_title").find("h2")
    news_date = soup.find("span", class_="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME")['data-date-time']
    # print(news_date.split(" ")[0])
    news_content = soup.find("article", class_="go_trans _article_content")

    word_counts = count_word(news_content.text, word_counts)

print(word_counts)

Counter({'증시': 68, '시장': 49, '지수': 44, '일': 44, '것': 38, '원': 37, '유가': 37, '등': 36, '연휴': 34, '코스피': 32, '년': 30, '하락': 30, '설': 28, '급락': 28, '글로벌': 28, '미국': 25, '선': 24, '장': 23, '포인트': 22, '월': 22, '거래일': 21, '일본': 21, '악재': 20, '달러': 19, '중국': 19, '수': 18, '국제': 17, '이후': 17, '마감': 16, '유럽': 16, '기업': 16, '금융': 16, '개장': 15, '한국': 14, '북한': 14, '경제': 14, '기자': 14, '홍콩': 13, '경기': 13, '낙폭': 12, '기간': 12, '금리': 12, '가능': 12, '증권': 12, '코스닥': 12, '주': 12, '이날': 12, '점': 11, '주가': 11, '말': 11, '만': 11, '개월': 10, '때': 10, '위기': 10, '무단': 10, '전재': 10, '배포': 10, '금지': 10, '기록': 10, '분석': 10, '외국인': 10, '주식': 10, '환율': 9, '중단': 9, '인상': 9, '리스크': 9, '우려': 9, '대비': 9, '기준': 9, '최대': 8, '개성': 8, '공단': 8, '폭락': 8, '요인': 8, '의장': 8, '영향': 8, '반영': 8, '서울': 8, '투자': 8, '연구원': 8, '엔화': 8, '거래': 8, '전망': 8, '미사일': 7, '발사': 7, '변수': 7, '개성공단': 7, '입주': 7, '시': 7, '심리': 7, '변동': 7, '발생': 6, '가동': 6, '사태': 6, '국내': 6, '불안': 6, '은행': 6, '재닛 옐런': 6, '둔화': 6, '뉴스': 6, '서울경제': 6, '시작': 6, '가운데': 6, '

In [176]:
forbidden_words = ["등", "것", "년","월", "일","이날"]

for word in list(word_counts.keys()):
    if word in forbidden_words:
        del word_counts[word]

In [177]:
word_counts

Counter({'증시': 68,
         '시장': 49,
         '지수': 44,
         '원': 37,
         '유가': 37,
         '연휴': 34,
         '코스피': 32,
         '하락': 30,
         '설': 28,
         '급락': 28,
         '글로벌': 28,
         '미국': 25,
         '선': 24,
         '장': 23,
         '포인트': 22,
         '거래일': 21,
         '일본': 21,
         '악재': 20,
         '달러': 19,
         '중국': 19,
         '수': 18,
         '국제': 17,
         '이후': 17,
         '마감': 16,
         '유럽': 16,
         '기업': 16,
         '금융': 16,
         '개장': 15,
         '한국': 14,
         '북한': 14,
         '경제': 14,
         '기자': 14,
         '홍콩': 13,
         '경기': 13,
         '낙폭': 12,
         '기간': 12,
         '금리': 12,
         '가능': 12,
         '증권': 12,
         '코스닥': 12,
         '주': 12,
         '점': 11,
         '주가': 11,
         '말': 11,
         '만': 11,
         '개월': 10,
         '때': 10,
         '위기': 10,
         '무단': 10,
         '전재': 10,
         '배포': 10,
         '금지': 10,
         '기록': 10